# Convolutional wordvec

## Reading the data

In [1]:
import pandas as pd
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')

In [2]:
wordvec_train = pd.read_pickle('train_wordvec.pickle')
wordvec_test = pd.read_pickle('test_wordvec.pickle')

In [3]:
train = train.merge(wordvec_train, on=['id'])
test = test.merge(wordvec_test, on=['id'])
train.head()

,id,keyword,location,text,target,cleaned_text,wordvec,keyword_wordvec,wordvec_concat,wordvec_tfidf
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[-0.26623327, 0.05843069, -0.1404636, -0.05265...","[-0.26623327, 0.05843069, -0.1404636, -0.05265...","[[-0.2820900082588196, 0.1519400030374527, -0....","[-2.0410312242232838, 0.1577752003302941, -0.8..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,"[-0.025449565, 0.031005142, -0.15566371, -0.23...","[-0.025449565, 0.031005142, -0.15566371, -0.23...","[[0.3039900064468384, 0.20476000010967255, -0....","[-0.27185601989428204, 0.2042857458194097, -1...."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,"[0.0059339865, 0.016337818, -0.105279535, -0.0...","[0.0059339865, 0.016337818, -0.105279535, -0.0...","[[0.00997759960591793, -0.20995000004768372, -...","[0.07528745450756767, 0.11175614595413208, -0...."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...,"[-0.18147185, 0.20731743, 0.014147284, -0.2182...","[-0.18147185, 0.20731743, 0.014147284, -0.2182...","[[-0.19686000049114227, 0.1157900020480156, -0...","[-1.3403782035623277, 1.2000715562275477, 0.11..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[-0.06394094, -0.01423019, 0.0063574947, 0.071...","[-0.06394094, -0.01423019, 0.0063574947, 0.071...","[[-0.02556299977004528, 0.444240003824234, -0....","[-0.7245167245467504, -0.364056259393692, 0.52..."


## Train a model

In [4]:
max_words = train['wordvec_concat'].apply(lambda x: x.shape[0]).max()

In [5]:
import numpy
def get_X(df, col):
    X = concat = numpy.empty((0, max_words, 300))
    for index, row in df.iterrows():
        x = numpy.pad(row[col],((0,max_words - row[col].shape[0]),(0, 0)))
        X = numpy.append(X, [x], axis=0)
    return X

In [6]:
y = train['target']
X = get_X(train, 'wordvec_concat')

In [7]:
X.shape

(7561, 32, 300)

In [12]:
import tensorflow as tf

def get_model():
    conv_model = tf.keras.Sequential([\
        tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),                           
        tf.keras.layers.Dense(units=1, activation='sigmoid'),
    ])
    return conv_model

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [14]:
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback
import numpy as np

class Metrics(Callback):
    def __init__(self, train, validation):   
        super(Metrics, self).__init__()
        self.validation = validation    
        self.train = train        
        
    def on_train_begin(self, logs={}):        
        self.val_f1s = []
        self.train_f1s = []
             
    def on_epoch_end(self, epoch, logs={}):
        val_targ = self.validation[1]   
        val_predict = (np.asarray(self.model.predict(self.validation[0]))).round()        
        
        train_targ = self.train[1]   
        train_predict = (np.asarray(self.model.predict(self.train[0]))).round()   
        
        val_f1 = f1_score(val_targ, val_predict)
        train_f1 = f1_score(train_targ, train_predict)
        self.val_f1s.append(round(val_f1, 6))
        self.train_f1s.append(round(train_f1, 6))
        
        print(f'— train_f1: {train_f1} — val_f1: {val_f1}')

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(5, random_state=42, shuffle=True)

for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(X=X, y=y)):
    print('---- Starting fold %d ----'%(k_fold+1))
    
    x_train, y_train = X[tr_inds], y[tr_inds]
    x_val, y_val = X[val_inds], y[val_inds]
    conv_model = get_model()
    conv_model.compile(loss='binary_crossentropy', optimizer= "adam", metrics=[])         
    conv_model.fit(x=x_train, y=y_train, batch_size=32, epochs=100, 
               callbacks=[Metrics(train=(x_train,y_train), validation=(x_val, y_val))])

---- Starting fold 1 ----
Epoch 1/100
189/189 [==============================] - 4s 20ms/step - loss: 0.5163
Epoch 2/100
189/189 [==============================] - 5s 24ms/step - loss: 0.4028
Epoch 3/100
189/189 [==============================] - 5s 25ms/step - loss: 0.3503
Epoch 4/100
  3/189 [..............................] - ETA: 3s - loss: 0.2512